# Cleaning and Organizing the Data from U.S.G.S.


Throughout the data wrangling process, we utilized the Pandas package. The dataset was queried on the USGS' website: https://sfbay.wr.usgs.gov/access/wqdata/index.html . This website provides extensive data on the water quality of the San Francisco Bay and lays out the entirety of the data that we analyzed. The site contains years of information for which we selected a subset of years that are more recent and relevant (2014 - 2018). Each querry is based on a year, such that a querry was made for water quality in the year 2014, water quality in the year 2015, and so on. Afterwards, the query was completed with a download of its excel file. For my convenience, I had created one excel file, "Main Data", that contains 4 sheets: 2014-2015, 2016, 2017, and 2018 water quality data.



In [2]:
import pandas as pd
data = pd.ExcelFile("SFBay_WaterQuality2014-208.xlsx")
tabnames = data.sheet_names

quality_201415 = data.parse(sheet_name = tabnames[0], skiprows = 0)
quality_2016 = data.parse(sheet_name = tabnames[1], skiprows = 0)
quality_2017 = data.parse(sheet_name = tabnames[2], skiprows = 0)
quality_2018 = data.parse(sheet_name = tabnames[3], skiprows =0)

Let's print the head of the "quality_2018" data frame. There are 22 columns in the dataset. Furthermore, some of the units of the columns (let's refer them to constituents) are placed in another row. Let's clean that up.

In [2]:
quality_2018.head()

,Date,Station Number,Depth,Discrete Chlorophyll,Chlorophyll a/a+PHA,Fluorescence,Calculated Chlorophyll,Discrete Oxygen,Oxygen Electrode Output,Oxygen Saturation %,...,Optical Backscatter,Calculated SPM,Measured Extinction Coefficient,Salinity,Temperature,Nitrite,Nitrate + Nitrite,Ammonium,Phosphate,Silicate
0,MM/DD/YYYY,NaN,[meters],[mg/m3],NaN,[volts],[mg/m3],[mg/L],[volts],NaN,...,[volts],[mg/L],[per meter],[psu],[°C],[µM],[µM],[µM],[µM],[µM]
1,2018-01-10 00:00:00,36.0,1,NaN,NaN,0.55,4.9,NaN,8.2,88.0,...,2.39,98,NaN,23.4,11.96,NaN,NaN,NaN,NaN,NaN
2,2018-01-10 00:00:00,36.0,2,4.2,0.51,0.46,4.1,NaN,8.2,89.0,...,2.6,107,NaN,23.54,11.97,2.66,76.39,13.42,6.97,110.84
3,2018-01-10 00:00:00,36.0,3,NaN,NaN,0.43,3.9,NaN,8.2,89.0,...,3.01,124,NaN,24.26,11.96,NaN,NaN,NaN,NaN,NaN
4,2018-01-10 00:00:00,36.0,4,NaN,NaN,0.46,4.1,NaN,8.2,89.0,...,5.12,213,NaN,24.36,11.97,NaN,NaN,NaN,NaN,NaN


We have found our first issue! The first row displays the corresponding units of measurement for each of the constitutents above it (note that some constituents may already have their corresponding units of measurement in their labels). We wish to rename the column labels by adding the corresponding name of unit of measurement with their associated constituent. For example, the column 'Depth' has the unit of measurement '[meters]'; we want this to become 'Depth [meters]' as the new column label. The method below will correct this issue by merging the first two rows.  

In [3]:
def unit_measurement_fix(df):
    cols = df.columns
    cols1 = [str(x) for x  in df.iloc[0,:]]
    cols1 = [(x,y) for x,y in zip(cols, cols1)]
    cols1 = [x + " " + y for x,y in cols1 ]
    cols1 = [x.split(" ") if "nan" in x else x for x in cols1 ]
    cols1 = [x[0] + " " + x[1] if isinstance(x, list) else x for x in cols1]
    df.columns = cols1
    df.drop(df.index[0], axis = 0, inplace=True)
    df.set_index(df.columns[0], inplace = True)
    return df


quality_201415 = unit_measurement_fix(quality_201415)
quality_2016 = unit_measurement_fix(quality_2016)
quality_2017 = unit_measurement_fix(quality_2017)
quality_2018 = unit_measurement_fix(quality_2018)

Let's examine the column names of the quality_2018 to see results of the unit_measurement_fix method:

In [5]:
quality_2018.columns

Index(['Station Number', 'Depth [meters]', 'Discrete Chlorophyll [mg/m3]',
       'Chlorophyll a/a+PHA', 'Fluorescence [volts]',
       'Calculated Chlorophyll [mg/m3]', 'Discrete Oxygen [mg/L]',
       'Oxygen Electrode Output [volts]', 'Oxygen Saturation',
       'Calculated Oxygen [mg/L]', 'Discrete SPM [mg/L]',
       'Optical Backscatter [volts]', 'Calculated SPM [mg/L]',
       'Measured Extinction Coefficient [per meter]', 'Salinity [psu]',
       'Temperature [°C]', 'Nitrite [µM]', 'Nitrate + Nitrite [µM]',
       'Ammonium [µM]', 'Phosphate [µM]', 'Silicate [µM]'],
      dtype='object')

Oneof the column names for the 2014/2015 dataset is incorrect: 'Depth ' should be 'Depth [meters]', This could have possibly been due to inconsistencies from the original dataset. 

In [6]:
quality_201415.columns

Index(['Station Number', 'Depth  ', 'Discrete Chlorophyll [mg/m3]',
       'Chlorophyll a/a+PHA', 'Fluorescence [volts]',
       'Calculated Chlorophyll [mg/m3]', 'Discrete Oxygen [mg/L]',
       'Oxygen Electrode Output [volts]', 'Oxygen Saturation',
       'Calculated Oxygen [mg/L]', 'Discrete SPM [mg/L]',
       'Optical Backscatter [volts]', 'Calculated SPM [mg/L]',
       'Measured Extinction Coefficient [per meter]', 'Salinity [psu]',
       'Temperature [°C]', 'Nitrite [µM]', 'Nitrate + Nitrite [µM]',
       'Ammonium [µM]', 'Phosphate [µM]', 'Silicate [µM]'],
      dtype='object')

In [8]:
# We'll use the quality_2018 dataframe column names as the replacement names. 
quality_201415.columns = [column_name for column_name in quality_2018.columns]
quality_201415.columns

Index(['Station Number', 'Depth [meters]', 'Discrete Chlorophyll [mg/m3]',
       'Chlorophyll a/a+PHA', 'Fluorescence [volts]',
       'Calculated Chlorophyll [mg/m3]', 'Discrete Oxygen [mg/L]',
       'Oxygen Electrode Output [volts]', 'Oxygen Saturation',
       'Calculated Oxygen [mg/L]', 'Discrete SPM [mg/L]',
       'Optical Backscatter [volts]', 'Calculated SPM [mg/L]',
       'Measured Extinction Coefficient [per meter]', 'Salinity [psu]',
       'Temperature [°C]', 'Nitrite [µM]', 'Nitrate + Nitrite [µM]',
       'Ammonium [µM]', 'Phosphate [µM]', 'Silicate [µM]'],
      dtype='object')

Now we've done the first step of the data cleansing process! Next, we will add the coordinates for the station numbers. Data of the coordinates can also be obtained from the USGS website. It was provided as an excel file, and here we import it and merge + map it with each of yearly data sets.

In [14]:
# I put the dataframe years inside a list. 
dataframes = [quality_201415, quality_2016, quality_2017, quality_2018]

#Column names: 'Station Number', 'North Longitude Degrees', 'North Latitude Minutes', 'West Longitude #Degrees','West Longitude Minutes', & 'Comments'
StationLocations = pd.read_csv("SFBay_TableofStationLocations.csv")

#Create a list of tuples, each containing the coordinates: (NL Deg NL Min, WL Deg, WL Min)
DMM_coordinates = [(str(x)+ " " + str(y), str(z) + " " + str(a)) for x, y, z, a in zip(StationLocations[StationLocations.columns[1]], StationLocations[StationLocations.columns[2]], StationLocations[StationLocations.columns[3]], StationLocations[StationLocations.columns[4]])]

#Create a dictionary where the station number is the key, coordinate of the associated section is the value. 
DMM_dict = {int(Station): Coordinates for Station,Coordinates in zip(StationLocations[StationLocations.columns[0]], DMM_coordinates)}

for df in dataframes:
    df['Coordinates'] = df["Station Number"].map(DMM_dict)

#Tidying up the order of column labels 
columns_list = list(df.columns)
columns_list.insert(2, columns_list[21])
del columns_list[22]

In [7]:
columns_list

['Station Number',
 'Depth [meters]',
 'Coordinates',
 'Discrete Chlorophyll [mg/m3]',
 'Chlorophyll a/a+PHA',
 'Fluorescence [volts]',
 'Calculated Chlorophyll [mg/m3]',
 'Discrete Oxygen [mg/L]',
 'Oxygen Electrode Output [volts]',
 'Oxygen Saturation',
 'Calculated Oxygen [mg/L]',
 'Discrete SPM [mg/L]',
 'Optical Backscatter [volts]',
 'Calculated SPM [mg/L]',
 'Measured Extinction Coefficient [per meter]',
 'Salinity [psu]',
 'Temperature [°C]',
 'Nitrite [µM]',
 'Nitrate + Nitrite [µM]',
 'Ammonium [µM]',
 'Phosphate [µM]',
 'Silicate [µM]']

In [18]:
# Rearrange for all dataframes:
for df in dataframes:
    df = df[columns_list]

We then stack the columns and make station number, depth, coordinates, and constituent combined into one index with each uniqie value. 

In [28]:
def stack_columns(df):
    df.reset_index(inplace = True)
    # Station Number, Depth [meters], coordinates, and constituent will become an index. 
    multi_df = df.set_index(list(df.columns[:4]))
    stacked_df = multi_df.stack(dropna = False).reset_index()
    # Sort the dataframe by Station Number and Depth [meters].
    stacked_df = stacked_df.sort_values(by = [stacked_df.columns[0], stacked_df.columns[1]])
    stacked_df.set_index(stacked_df.columns[0], inplace = True)
    return stacked_df

In [31]:
for df in dataframes:
    df = stack_columns(df).rename(columns = {0:'Value', 'level_4': 'Constituent'})

Finally, we concatenate the modified dataframes together:

In [32]:
water_quality = pd.concat([quality_201415,quality_2016, quality_2017, quality_2018])
# Let's examine the rows
water_quality.head()

,Station Number,Depth [meters],Coordinates,Constituent,Value
Date MM/DD/YYYY,,,,,
2014-01-14,2.0,1,"(38 3.8', -121 51.1')",Discrete Chlorophyll [mg/m3],NaN
2014-01-14,2.0,1,"(38 3.8', -121 51.1')",Chlorophyll a/a+PHA,NaN
2014-01-14,2.0,1,"(38 3.8', -121 51.1')",Fluorescence [volts],0.22
2014-01-14,2.0,1,"(38 3.8', -121 51.1')",Calculated Chlorophyll [mg/m3],1.9
2014-01-14,2.0,1,"(38 3.8', -121 51.1')",Discrete Oxygen [mg/L],NaN


We now have a workable dataframe that we can perform some analysis on! 